In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load HMGP Dataset

In [2]:
buyout_props = pd.read_csv('../../../Local_Data/FEMA_HMGP/HazardMitigationAssistanceMitigatedProperties.csv')
buyout_props = buyout_props.dropna(subset=['disasterNumber'])
buyout_props['disasterNumber'] = buyout_props['disasterNumber'].astype(int)

/tmp/ipykernel_3587/2063840798.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  buyout_props = pd.read_csv('../../../Local_Data/FEMA_HMGP/HazardMitigationAssistanceMitigatedProperties.csv')


# FIPS mapping

In [3]:
# Read the file line by line
with open('../../../Local_Data/fips_codes.txt', 'r') as file:
    lines = file.readlines()

# Step 1: Identify where state-level data starts and ends
state_section_start = 0
county_section_start = 0

for i, line in enumerate(lines):
    if "state-level" in line.lower():
        state_section_start = i + 2  # State data starts after the "state-level" heading
    if "county-level" in line.lower():
        county_section_start = i + 2  # County data starts after the "county-level" heading
        break

# Manually create the dictionary mapping state FIPS codes to state names
state_fips_dict = {
    "01": "ALABAMA",
    "02": "ALASKA",
    "04": "ARIZONA",
    "05": "ARKANSAS",
    "06": "CALIFORNIA",
    "08": "COLORADO",
    "09": "CONNECTICUT",
    "10": "DELAWARE",
    "11": "DISTRICT OF COLUMBIA",
    "12": "FLORIDA",
    "13": "GEORGIA",
    "15": "HAWAII",
    "16": "IDAHO",
    "17": "ILLINOIS",
    "18": "INDIANA",
    "19": "IOWA",
    "20": "KANSAS",
    "21": "KENTUCKY",
    "22": "LOUISIANA",
    "23": "MAINE",
    "24": "MARYLAND",
    "25": "MASSACHUSETTS",
    "26": "MICHIGAN",
    "27": "MINNESOTA",
    "28": "MISSISSIPPI",
    "29": "MISSOURI",
    "30": "MONTANA",
    "31": "NEBRASKA",
    "32": "NEVADA",
    "33": "NEW HAMPSHIRE",
    "34": "NEW JERSEY",
    "35": "NEW MEXICO",
    "36": "NEW YORK",
    "37": "NORTH CAROLINA",
    "38": "NORTH DAKOTA",
    "39": "OHIO",
    "40": "OKLAHOMA",
    "41": "OREGON",
    "42": "PENNSYLVANIA",
    "44": "RHODE ISLAND",
    "45": "SOUTH CAROLINA",
    "46": "SOUTH DAKOTA",
    "47": "TENNESSEE",
    "48": "TEXAS",
    "49": "UTAH",
    "50": "VERMONT",
    "51": "VIRGINIA",
    "53": "WASHINGTON",
    "54": "WEST VIRGINIA",
    "55": "WISCONSIN",
    "56": "WYOMING"
}

# Convert the dictionary to a DataFrame for easy merging
state_df = pd.DataFrame(list(state_fips_dict.items()), columns=['State FIPS', 'State Name'])

# Step 1: Split the data into sections
state_section_start = 0
county_section_start = 0

for i, line in enumerate(lines):
    if "state-level" in line.lower():
        state_section_start = i + 2  # Skip the header lines
    if "county-level" in line.lower():
        county_section_start = i + 2  # Skip the header lines
        break

state_lines = lines[state_section_start:county_section_start-2]  # State data lines
county_lines = lines[county_section_start:]  # County data line

# Step 3: Parse county data
county_fips = []
county_name = []

for line in county_lines:
    line = line.strip()
    if len(line) > 12 and line[:5].isdigit():
        county_fips_code = line[:5].strip()
        county_fips_name = line[12:].strip()
        county_fips.append(county_fips_code)
        county_name.append(county_fips_name)

county_df = pd.DataFrame({
    'countyCode': county_fips,
    'County Name': county_name
})

county_df['State FIPS'] = county_df['countyCode'].str[:2]  # Extract state FIPS from the countyCode

# Assuming county_df is already created and correctly populated:
fips_df = county_df.merge(state_df, on='State FIPS')
fips_df['County Name'] = fips_df['County Name'].str.upper()

In [4]:
# Ensure the countyCode is a string with leading zeros if needed
buyout_props = buyout_props.dropna(subset=['county', 'state'])
buyout_props['county'] = buyout_props['county'].str.upper()
buyout_props['state'] = buyout_props['state'].str.upper()

# Remove "COUNTY" from the 'County Name' in fips_df
fips_df['County Name'] = fips_df['County Name'].str.replace(' COUNTY', '', regex=False).str.replace(' PARISH', '', regex=False)

# Now perform the merge
buyout_props = buyout_props.merge(fips_df, left_on=['county', 'state'], right_on=['County Name', 'State Name'], how='left')

# Load and merge disasters dataset

In [7]:
filtered_disasters = pd.read_csv('../../final_filtered_disasters.csv')
filtered_disasters['fullFIPS'] = filtered_disasters['fullFIPS'].astype(str).str.zfill(5)
filtered_disasters['disasterNumber'] = filtered_disasters['disasterNumber'].astype(int)

In [8]:
# Identify columns that are in both dataframes except for 'disasterNumber'
common_columns = list(set(buyout_props.columns).intersection(set(filtered_disasters.columns)))
common_columns.remove('disasterNumber')  # Remove 'disasterNumber' from the list of common columns

# Select only the columns from filtered_disasters that are not in buyout_props, except for 'disasterNumber'
filtered_disasters = filtered_disasters.drop(columns=common_columns)

In [9]:
# Merging the result_df with disasters_df on 'countyCode' (from result_df) and 'fullFIPS' (from disasters_df),
# and 'disasterNumber'
merged_df = buyout_props.merge(filtered_disasters, left_on=['countyCode', 'disasterNumber'], 
                               right_on=['fullFIPS', 'disasterNumber'], how='inner')


# Print the number of rows before the merge
print(f"Number of rows in buyout_props before the merge: {len(buyout_props)}")

# Dropping the duplicate 'countyCode' column after the merge as it duplicates 'fullFIPS'
merged_df = merged_df.drop(columns=['fullFIPS'])

# Print the number of rows before the merge
print(f"Number of rows in buyout_props after the merge (only flood-related grants): {len(merged_df)}")

Number of rows in buyout_props before the merge: 80994
Number of rows in buyout_props after the merge (only flood-related grants): 46782


In [10]:
flood_buyouts_only = merged_df[
    merged_df['propertyAction'].isin(['Acquisition/Demolition', 'Acquisition/Relocation'])
]

print(f"Total buyouts: {len(flood_buyouts_only)}")

Total buyouts: 17029


In [11]:
merged_df.to_csv("final_filtered_HMGP.csv")
flood_buyouts_only.to_csv("final_filtered_buyouts.csv")